In [2]:
decoder = Decoder()
with open('cloverleaf.pbf', 'rb') as f:
    encoded = f.read()
ret = decoder.decode(encoded)

print(type(ret), len(ret))
assert ret == decoder.decode(encoded)

<class 'str'> 365500


In [3]:
ret = decoder.decode_to_rapidjson(encoded)
print(type(ret), type(ret()))

<class '_pybind11_geobuf.rapidjson'> <class 'dict'>


In [4]:
geojson = decoder.decode_to_rapidjson(encoded)()
assert geojson['type'] == 'FeatureCollection'

In [5]:
assert 8 == decoder.precision()

In [6]:
encoder = Encoder(max_precision=10**8)
encoded = encoder.encode({
    "type": "Polygon",
    "coordinates": [[[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]]]
})

print(encoded)

from pybind11_geobuf import pbf_decode
print(pbf_decode(encoded))

b'\x18\x002\r\x08\x04\x1a\t\xc8\x01\x00\x02\x00\x00\x02\x01\x00'
3: 0
6: 
  1: 4
  3: 200,0,2,0,0,2,1,0



In [7]:
print(decoder.decode_to_rapidjson(encoded)())

{'type': 'Polygon', 'coordinates': [[[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]]]}
